# R/C Implementation of a Scalar Bekk Model
> Moritz Degler, Lorenzo Pisati, Nandan Rao, Jonas Paul Westermann

## Implementation Notes
### Parameter convention:
The model uses two parameters:
* $\alpha$: the factor of the lagged $y^2$
* $\beta$: the factor of the lagged sigma
However, since the optimisation libraries only support box constraints, a trick must be used to keep not only $\alpha$ and $\beta$ between 0 and 1, but also $0 \leq \alpha+\beta \leq 1$. The way to do this, is to feed two parameters into the filter, $\gamma$ and $\lambda$ such that:
$$
\alpha = (1-\lambda) * \gamma \\
\beta = (1-\lambda) * (1 - \gamma)
$$
and then we constrain these two parameters to $(0,1)$. This results in the overall step for an update looking as follows:
$$
S_t = \lambda C + \sum_{k=1}^K \frac{\alpha}{k} y_{t-k}y_{t-k}^{T} + \frac{\beta}{k} S_{t-k}
$$

### Sigma Initialisation
$S_0$ is initialized at the unconditional covariance, so is $C$, the constant term in the model. 

### MLE
MLE is used to compute the parameters, assuming in this case that the de-meaned returns (y) follow a multivariate normal distribution, with a covariance matrix that is predicted via the model. Because of this, one needs to both update the covariance matrix via the model and compute the individual likelihood of every observation (every t in T of the time-series dataset). Because of THIS, we decompose $S_t$ into its Cholesky Factorization, which we update for every t in T and use directly in the computation of the log likelihood (from the cholesky we have both a cheap inverse, via forward substitution, and a cheap determinant, via the diagonal). 

Given a cholesky decomposition, one can update the matrix directly as long as the update is the addition or subtraction of a rank-1 matrix. Clearly the $y_{t-k} y^T_{t-k} $ is a rank-1 matrix, as it is an outer product. This is the update used in a certain similar algorithm whose most noteworthy R implimentation is referred to as MEWMA. In order to make the most uniform API and usable abstractions for parameter updating across multiple models, including MEWMA, we utilize this rank-1 updating in this algorithm as well. To do so, we decompose $C$ into its Cholesky factorization, then use the outer-product of each column of the decomposition as a rank-1 update. This maintains the complexity of every loglikelihood calculation at every t in T being $O(N^3)$. 

## Examples

In [3]:
dyn.load('./C/bekk_log_lik.so')
source('./R/bekk_model.R')

In [4]:
y <- read.csv('../ganter/data/Problemsets Data (Tickers)/all.csv')
y <- apply(y,c(2),diff)
y[is.na(y)] <- 0

#### Single Filter Pass

In [1]:
scalar.bekk.filter(as.matrix(y[1:10,2:4]),c(0.9,0.5),k=1)

ERROR: Error in eval(expr, envir, enclos): could not find function "scalar.bekk.filter"


#### Different Lags

In [4]:
print('With only 1 lag.')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list())[c('param','obj')]
print('With 3 lags')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list(lags = 3))[c('param','obj')]
print('With 5 lags')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list(lags = 5))[c('param','obj')]

[1] "With only 1 lag."


$param
     alpha       beta 
0.01959369 0.10338052 

$obj
[1] -18695.48

[1] "With 3 lags"


$param
     alpha       beta 
0.05796592 0.32341340 

$obj
[1] -24470.29

[1] "With 5 lags"


$param
    alpha      beta 
0.0635357 0.3464407 

$obj
[1] -23895.7

#### Large Dataset

In [9]:
print("25 time-series with T=4025")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 1))[c('param','obj')])
print("50 time-series with T=4025")
system.time(scalar.bekk.fit(as.matrix(y[,1:50]),opts=list(lags = 1))[c('param','obj')])
print("80 time-series with T=4026")
system.time(scalar.bekk.fit(as.matrix(y),opts=list(lags = 1))[c('param','obj')])

[1] "25 time-series with T=4025"


   user  system elapsed 
 13.298   0.941  14.254 

[1] "50 time-series with T=4025"


   user  system elapsed 
 82.648   3.975  86.909 

[1] "80 time-series with T=4026"


   user  system elapsed 
391.913  17.854 408.662 

#### Large Data and more Lags

In [10]:
print("25 time-series with T=4025 and three lags")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 3))[c('param','obj')])
print("25 time-series with T=4025 and five lags")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 5))[c('param','obj')])

[1] "25 time-series with T=4025 and three lags"


   user  system elapsed 
 22.309   1.008  23.373 

[1] "25 time-series with T=4025 and five lags"


   user  system elapsed 
 33.669   0.938  34.645 

#### Alternate Optimisation Library

In [5]:
print("25 time-series with T=4025 and three lags and using nlminb")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 3, optim.lib="nlminb"))[c('param','obj')])

[1] "25 time-series with T=4025 and three lags and using nlminb"


Warning message in nlminb(param.init, obj, lower = 0, upper = 1, k = lags):
“NA/NaN function evaluation”

   user  system elapsed 
 36.740   1.484  39.245 